In [5]:
import numpy as np
import threading
import queue
testArray = np.array([[1, 2], [2, 1]])
np.shape(testArray)

(2, 2)

In [6]:
def simple_NMF(A, k, i):
    m, n = np.shape(A)
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    for _ in range(i):
        W = W * (A @ H.T) / (W @ H @ H.T)
        H = H * (W.T @ A) / (W.T @ W @ H)
    return W @ H

simple_NMF(testArray, 2, 1000)

array([[1., 2.],
       [2., 1.]])

In [7]:
def thread_function_w(a, w, h, q, i):
    q.put((i, w * (a @ h.T) / (w @ h @ h.T)))

def thread_function_h(a, w, h, q, i):
    q.put((i, h * (w.T @ a) / (w.T @ w @ h)))

def naive_parallel_NMF(a, k, p, numIter):
    m, n = np.shape(a)
    if m % p > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % p > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    w = np.random.rand(m, k)
    h = np.random.rand(k, n)
    a_pieces_1 = [a[int(i*m/p):int((i+1)*m/p), :] for i in range(p)]
    a_pieces_2 = [a[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)]
    
    for _ in range(numIter):
        w_pieces = [w[int(i*m/p):int((i+1)*m/p), :] for i in range(p)] # chop up w
        w_threads = []
        w_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_w, args = (a_pieces_1[j], w_pieces[j], h, w_queue, j)) # each thread updates one section
            newThread.start()
            w_threads.append(newThread)
        for thread in w_threads: # wait for all threads to complete
            thread.join()
        while not w_queue.empty(): # reconstitute and update w
            i, v = w_queue.get()
            w_pieces[i] = v
        w = np.concatenate(w_pieces, 0)

        h_pieces = [h[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)] # same procedure for h
        h_threads = []
        h_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_h, args = (a_pieces_2[j], w, h_pieces[j], h_queue, j))
            newThread.start()
            h_threads.append(newThread)
        for thread in h_threads:
            thread.join()
        while not h_queue.empty():
            i, v = h_queue.get()
            h_pieces[i] = v
        h = np.concatenate(h_pieces, 1)
    return w @ h
    
naive_parallel_NMF(testArray, 2, 2, 10000)

array([[1., 2.],
       [2., 1.]])

In [52]:
def vertical_thread_function_u(h, q):
    q.put(h @ h.T)

def vertical_thread_function_v(a, h, q, i, j):
    q.put((i, j, a @ h.T))

def vertical_thread_function_x(w, q, i, j):
    q.put((i, j, w.T @ w))

def vertical_thread_function_y(a, w, q, i, j):
    q.put((i, j, w.T @ a))

def vertical_HPC_NMF(a, k, p1, p2, numIter):
    m, n = np.shape(a)
    if m % (p1*p2) > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % (p1*p2) > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    w = np.random.rand(m, k)
    h = np.random.rand(k, n)

    a_pieces = [[a[int(i*n/p1):int((i+1)*n/p1), int(j*n/p2):int((j+1)*n/p2)] for i in range(p1)] for j in range(p2)]
    print(np.shape(a_pieces))

    for _ in range(numIter):
        u = np.zeros((k, k))
        h_pieces_u = [[np.random.rand(k, int(n/(p1*p2))) for i in range(p1)] for j in range(p2)] # chop up h into full grid #TO DO
        threads_u = [[] for _ in range(p2)]
        thread_queue_u = queue.Queue()
        for i in range(p2): # keep threads in a p1 by p2 grid, like the paper does
            for j in range(p1):
                newThread = threading.Thread(target = vertical_thread_function_u, args = (h_pieces_u[i][j], thread_queue_u))
                newThread.start()
                threads_u[i].append(newThread)
        for thread in [item for sublist in threads_u for item in sublist]:
            thread.join()
        while not thread_queue_u.empty():
            u += thread_queue_u.get()

        v_pieces = [np.zeros((m, int(k/p1))) for _ in range(p1)] # keep v split up into columns
        h_pieces_v = [np.random.rand(k, int(n/p2)) for j in range(p2)] # chop up h into columns (for calculation of y) # TO DO
        threads_v = []
        thread_queue_v = queue.Queue()
        for i in range(p2):
            newThread = threading.Thread(target = vertical_thread_function_y, args = (h_pieces_v[i], thread_queue_v, i, j))
            newThread.start()
            threads_v.append(newThread)
        for thread in threads_v:
            thread.join()
        while not thread_queue_v.empty():
            i, val = thread_queue_v.get()
            v_pieces[i] += val

        x = np.zeros((k, k))
        y = np.zeros((k, n))

    print(f"u:\n{u}\n\nv:\n{[]}")
    return

vertical_HPC_NMF(np.zeros((16, 8)), 2, 4, 2, 1)

Exception in thread Thread-80368:
Traceback (most recent call last):
  File "c:\Users\james\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-80369:
Traceback (most recent call last):
  File "c:\Users\james\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
        self.run()
  File "c:\Users\james\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
self.run()
  File "c:\Users\james\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
TypeError: vertical_thread_function_y() missing 1 required positional argument: 'j'
    self._target(*self._args, **self._kwargs)
TypeError: vertical_thread_function_y() missing 1 required positional argument: 'j'


(4, 16, 2)
u:
[[3.39512916 2.80669921]
 [2.80669921 2.89613317]]

v:
[]
